In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

from tqdm.auto import tqdm

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import numpy as np
import pandas as pd
import html
from pymystem3 import Mystem


ALLOWED_ALPHABET=list(map(chr, range(ord('а'), ord('я') + 1)))
ALLOWED_ALPHABET.extend(map(chr, range(ord('a'), ord('z') + 1)))
ALLOWED_ALPHABET.extend(list(map(str.upper, ALLOWED_ALPHABET)))

# ALLOWED NUMBERS
ALLOWED_ALPHABET.extend([str(x) for x in range(10)])

ALLOWED_ALPHABET = set(ALLOWED_ALPHABET)


def normalize(sentence):
    def validate_char(c):
        return c in ALLOWED_ALPHABET
    
    sentence = ''.join(map(lambda c: c if validate_char(c) else ' ', sentence.lower()))
    return ''.join(sentence).strip()

def collect(row, word_to_lemma, count, stemmer, correct_spelling):
    # Заменить html сущности на символы
    row = html.unescape(row)

    # Оставить только символы Аа_Яя Aa_Zz и привести в нижний регистр
    normalized = normalize(row)

    # Разделить предложения на слова по ' ', убрать пустые элементы, заменить опечатки
    list_row = list()
    for word in normalized.split(' '):
        if word != '':
            correct_form = correct_spelling.get(word, word)
            # Здесь одно слово может быть разбито на 2, "напримертакое" -> "например такое". Их нужно собрать
            for part in correct_form.split(' '):
                list_row.append(part)

    # Лемматизация слов и сохранение пары слово:лемма в словарь для ускорения работы.
    lemmatized = list()
    for word in list_row:
        if word not in word_to_lemma:
            word_to_lemma[word] = stemmer.lemmatize(word.strip())[0]
        lemma = word_to_lemma[word]

        if lemma in count:
            count[lemma] += 1
        else:
            count[lemma] = 1
        lemmatized.append(lemma)
    
    return ' '.join(lemmatized)

def keep_common(sent, counts, dictionary_threshold, unk_token, keep_order):
    common_words = [unk_token if counts[word] < dictionary_threshold else word for word in sent.split()]

    # Если отсортировать слова, то уберется много дубликатов в корпусе
    if not keep_order:
        common_words = sorted(common_words)
    
    return ' '.join(common_words)

def clean_price(data):
    # Много строк без информации и большой ценой
    data['name_dish'] = data['name_dish'].where(data.name_dish != 'Третий набор в подарок')
    data.dropna(inplace = True)

    # Дороже 100к
    data.price = data.price.apply(lambda x: x/1000 if x>100000 else x)

    # Дороже 15к и не сеты
    idx = data.loc[(data.price > 15000) & ~(data.category.isin(['Рыба и морепродукты','Морепродукты', 'Сеты', 'Комбо', 'Наборы','Торты', 'Мясо']))].index
    data['price'] = data['price'].where(~data.index.isin(idx), data['price']/100)

    # Оставшиеся сеты
    data['price'] = data['price'].apply(lambda x: x/100 if x>35000 else x)

    data = data.astype({'price': int})
    return data

def clean_words(data, dictionary_threshold, unk_token, keep_order, to_spell_path):
    stemmer = Mystem()

    # Словарь с заменой опечаток
    correct_spelling = np.load(to_spell_path, allow_pickle=True).item()
    
    # Нормализация и подсчет встречаемости слов
    word_to_lemma = dict()
    count_description = dict()
    count_name = dict()
    data['product_description'] = data['product_description'].apply(lambda x: collect(x, word_to_lemma, count_description, stemmer, correct_spelling))
    data['name_dish'] = data['name_dish'].apply(lambda x: collect(x, word_to_lemma, count_name, stemmer, correct_spelling))

    # Замена редких слов на <UNK>
    data['product_description'] = data['product_description'].apply(lambda x: keep_common(x, count_description, dictionary_threshold, unk_token, keep_order))
    data['name_dish'] = data['name_dish'].apply(lambda x: keep_common(x, count_name, dictionary_threshold, unk_token, keep_order))

    # Удаление дубликатов
    data.drop_duplicates(subset = ['category','name_dish','product_description','tags_menu'], inplace = True)
    return data

def clean_df(data, dictionary_threshold = 30, unk_token = '<UNK>', keep_order = True, to_spell_path = 'to_spell.npy'):
    data = data.copy()

    # Замена словаря в поле tags_menu на его значение
    data['tags_menu'] = data['tags_menu'].apply(lambda x: [k for k in x.keys()][0])

    # Удаление одной строки с пустыми name_dish, product_description, price
    data.dropna(inplace = True)
    data.reset_index(drop=True, inplace=True)

    # Очистка price
    data = clean_price(data)
    
    # Очистка name_dish, product_description
    data = clean_words(data, dictionary_threshold, unk_token, keep_order, to_spell_path)

    return data

In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
pd.set_option('display.max_rows', 500)

data = pd.read_pickle('menu_df_for_made.pkl')

data_c = clean_df(data, dictionary_threshold=5, unk_token='', keep_order = True)
data_c.price = data_c.price.apply(lambda x: str(int(x/100)*100))
data_c['X'] = '[NAME] ' + data_c['name_dish'] + ' [DESC] ' + data_c['product_description'] + ' [PRICE] ' + data_c['price']

enc = OrdinalEncoder()
data_c['y'] = enc.fit_transform(data_c.tags_menu.values.reshape(-1, 1))
data_c = data_c.astype({'y': int})

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_c['X'], data_c['y'], test_size=0.2, stratify = data_c['y'], random_state=42, shuffle = True)

In [ ]:
train = pd.DataFrame({'X':X_train, 'y': y_train})
test = pd.DataFrame({'X':X_test, 'y': y_test})

In [ ]:
df = pd.concat([train ,test], axis = 0, ignore_index = True)

In [ ]:
tweets = df.X.values
labels = df.y.values

In [ ]:
# Load the BERT tokenizer
# ADDING SPECIAL TOKENS HERE
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence', do_lower_case=True, additional_special_tokens = ['[NAME]', '[DESC]', '[PRICE]'])

In [ ]:
print(' Original: ', tweets[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(tweets[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweets[0])))

In [ ]:
max_len = 85

In [ ]:
input_ids = []
attention_masks = []

for tweet in tqdm(tweets):
    encoded_dict = tokenizer.encode_plus(
                        tweet,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', tweets[0])
print('Token IDs:', input_ids[0])

In [ ]:
X_train_idx, X_test_idx, y_train, y_test = train_test_split(range(len(input_ids)), labels, random_state = 42, stratify = labels, shuffle=True, test_size=0.2)

In [ ]:
train_dataset = TensorDataset(input_ids[X_train_idx, :], attention_masks[X_train_idx, :], y_train)
val_dataset = TensorDataset(input_ids[X_test_idx, :], attention_masks[X_test_idx, :], y_test)

In [ ]:
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size, 
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size*2 
        )

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased-sentence", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 39, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model = model.cuda()

In [ ]:
model.resize_token_embeddings(model.bert.embeddings.word_embeddings.num_embeddings + 3)

In [ ]:
class_sample_count = np.unique(df['y'].values, return_counts=True)[1]
weight = torch.FloatTensor(1. / class_sample_count)
weight = np.power(weight*1000000, 0.2)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8 
                )

criterion = nn.CrossEntropyLoss(weight = weight.to(device))

# Fine tuning the model

In [ ]:
epochs = 9

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print(f"  Last loss: {loss.item()}")
        
        if step % 8500 == 0 and not step == 0:
            # ========================================
            #               Validation
            # ========================================
            # After the completion of each training epoch, measure our performance on
            # our validation set.

            print("")
            print("Running Validation...")

            t0 = time.time()

            # Put the model in evaluation mode--the dropout layers behave differently
            # during evaluation.
            model.eval()

            # Tracking variables 
            best_eval_loss = float('+inf')
            total_eval_loss = 0
            nb_eval_steps = 0

            # Evaluate data for one epoch
            for batch in tqdm(validation_dataloader):

                # Unpack this training batch from our dataloader. 
                #
                # As we unpack the batch, we'll also copy each tensor to the GPU using 
                # the `to` method.
                #
                # `batch` contains three pytorch tensors:
                #   [0]: input ids 
                #   [1]: attention masks
                #   [2]: labels 
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)

                # Tell pytorch not to bother with constructing the compute graph during
                # the forward pass, since this is only needed for backprop (training).
                with torch.no_grad():        

                    # Forward pass, calculate logit predictions.
                    # token_type_ids is the same as the "segment ids", which 
                    # differentiates sentence 1 and 2 in 2-sentence tasks.
                    # The documentation for this `model` function is here: 
                    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                    # Get the "logits" output by the model. The "logits" are the output
                    # values prior to applying an activation function like the softmax.
                    output= model(b_input_ids, 
                                           token_type_ids=None, 
                                           attention_mask=b_input_mask,
                                           labels=b_labels)

                # Accumulate the validation loss.
                logits = output.logits
                loss = criterion(logits, b_labels)
                total_eval_loss += loss.item()

                # Move logits and labels to CPU
                #logits = logits.detach().cpu().numpy()
                #label_ids = b_labels.to('cpu').numpy()

                # Calculate the accuracy for this batch of test sentences, and
                # accumulate it over all batches.
                # total_eval_accuracy += flat_accuracy(logits, label_ids)


            # Report the final accuracy for this validation run.
            # avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
            # print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

            # Calculate the average loss over all of the batches.
            avg_val_loss = total_eval_loss / len(validation_dataloader)
            print(f'Average val loss: {avg_val_loss}')

            # Measure how long the validation run took.
            validation_time = format_time(time.time() - t0)

            if avg_val_loss < best_eval_loss:

                torch.save(model, f'{epoch_i}_bert_model')
                best_eval_loss = avg_val_loss
            #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            #print("  Validation took: {:}".format(validation_time))

            # Record all statistics from this epoch.
            #training_stats.append(
            #    {
            #        'epoch': epoch_i + 1,
            #        'Training Loss': avg_train_loss,
            #        'Valid. Loss': avg_val_loss,
            #        'Valid. Accur.': avg_val_accuracy,
            #        'Training Time': training_time,
            #        'Validation Time': validation_time
            #    }
            #)

            

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
        logits = output.logits
        loss = criterion(logits, b_labels)
        ########################
        #loss = output.loss
        ########################
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))